In [102]:
# when not using colab
import sys
sys.path.append('../src/QA')
import reload_recursive
%reload data_prep
from model import QA_RNN
%reload word_embed
from word_embed import WordEmbed
%reload embed_doc
from embed_doc import EmbedAndConcat

In [103]:
from data_prep import DataPrep
%reload model

All these functions should be moved to their own module, im just doing it like this as a rough draft.

In [41]:
def train(input_question_tensor, input_document_tensor, target_tensor, question_model, document_model, question_optimizer, document_optimizer, criterion):
    input_question_length = input_question_tensor.size(0)
    input_document_length = input_document_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0    
    question_optimizer.zero_grad()
    document_optimizer.zero_grad()

    # get output from question model
    question_output = question_model(input_question_tensor)
    
    # predictions from the document model with output from question for attention
    document_output = document_model(input_document_tensor, question_output)

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [42]:
import time
import math

# Helper functions for training
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [43]:
import random
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        random_choice_ix = random.choice(range(n_data)) # Get a random index within the scope of input data
        input_index_r = [[ind] for ind in input_index[random_choice_ix]] 
        target_index_r = [[ind] for ind in target_index[random_choice_ix]]
        
        input_tensor = torch.LongTensor(input_index_r).to(device) 
        target_tensor = torch.LongTensor(target_index_r).to(device) 

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0


In [44]:
import torch
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [104]:
df = DataPrep.parse_tsv('../WikiQA-train.tsv')
question_doc_raw_train = DataPrep.convert_pd_to_json(df)

In [142]:
# TODO - add to package somwhere
def get_unrolled_embeddings(df):
    q_inputs = []
    doc_inputs = []
    doc_targets = []
    for key in df.keys():
        toks = DataPrep.tokenize_question_and_doc(df[key])
        q_embeds = EmbedAndConcat.q_concat(toks[0])
        doc_embeds = EmbedAndConcat.doc_concat(toks[1])
        # toks[2] are the targets
        # unroll the document and tokens.
        # that is, create from [[sent1_word1_embed, sent1_word2_embed,...], [sent2_word1_embed...]]
        # simply [word1_embed, word2_embed...]
        unrolled_doc_embeds = [word_embed for sentence in doc_embeds for word_embed in sentence]
        unrolled_doc_targets = [target for sentence in toks[2] for target in sentence]
        q_inputs.append(q_embeds)
        doc_inputs.append(unrolled_doc_embeds)
        doc_targets.append(unrolled_doc_targets)
    return q_inputs, doc_inputs, doc_targets

train_question_inputs, train_doc_inputs, train_doc_targets = get_unrolled_embeddings(question_doc_raw_train)

In [165]:
def get_max(l):
    return max(len(item) for item in l)
def get_min(l):
    return min(len(item) for item in l)

In [155]:
print(get_max(train_question_inputs))
print(get_max(train_doc_inputs))
print(get_max(train_doc_targets))

27
1064
1064


Now do the same for the test set

In [160]:
df_test = DataPrep.parse_tsv('../WikiQA-test.tsv')
question_doc_raw_test = DataPrep.convert_pd_to_json(df_test)
test_q_inputs, test_doc_inputs, test_doc_targets = get_unrolled_embeddings(question_doc_raw_test)

In [161]:
print(get_max(test_q_inputs))
print(get_max(test_doc_inputs))
print(get_max(test_doc_targets))

21
1889
1889


apply padding

In [ ]:
q_len = max(get_max(train_q_inputs), get_max(test_q_inputs))
doc_len = max(get_max(train_doc_inputs), get_max(test_doc_inputs))
def apply_padding(q_len, doc_len, ) # just cut for now maxy boy. Or add np.zeros to fill?